In [1]:
import pandas as pd
import requests
import re

In [2]:
def crowl_content(id):
    url = 'https://www.dcard.tw/_api/posts/'+str(id)
    web = requests.get(url)
    jsn = web.json()
    return jsn['content']

In [ ]:


url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100'
print('start')
web = requests.get(url)

jsn = web.json()

content_list = []

for i in range(len(jsn)):
    print(i)
    ids = jsn[i]['id']
    content_list.append(crowl_content(ids))
    last = ids
    
for j in range(10):
    print(j)
    url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + str(last)
    web = requests.get(url)
    jsn = web.json()
    for i in range(len(jsn)):
        ids = jsn[i]['id']
        try:
            content_list.append(crowl_content(ids))
            last = ids
        except KeyError:
            last = ids

In [7]:
len(content_list)

114

In [8]:
import jieba
import pandas as pd
from collections import Counter
import math

In [9]:
jieba.load_userdict('testdic.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.187 seconds.
Prefix dict has been built succesfully.


In [10]:
def keyword(sentence):
#     cut the sentence with jieba
    article = sentence
    article_cut = jieba.lcut(article)
# only take words that length > 2
    filter_cut = []
    for w in article_cut:
        if len(w)>1:
            filter_cut.append(w)
# count 'tf' values
    c = Counter(filter_cut)
    total = len(set(filter_cut))
    df = pd.DataFrame(c.keys(),c.values())
    df['tf'] = df.index/total


# def log function
    def log(x):
        return math.log(x,10)

    total_content = len(content_list)
# count this word be metions in other articles times
    metion_list=[]
    for word in df[0]:
        metion = 0
        for a in content_list:
            if word in a:
                metion+=1
        if metion == 0:
            metion+=1
        metion_list.append(metion)
# count 'idf' values
    df['metion']=metion_list
    df['idf'] = df['metion'].apply(lambda x : log(total_content/x))
# count 'tfidf' values
    df['tfidf'] = df['tf']*df['idf']

    df = df.sort_values('tfidf',ascending=False)
    return df

In [14]:
kw = keyword('''專櫃粉底液請益 植村秀/雅詩蘭黛/蘭蔻
美妝1月30日 23:47
大家好～女生要對自己好一點😆
新年拿了紅包就想敗下去買新的粉底液
然而我有選擇障礙剛好又是第一瓶專櫃粉底所以遲遲下不了決定
想問問大家的意見～❣️

膚質👉🏻中性偏油肌（T字部位偏油） 臉上有一些毛孔、泛紅跟痘疤
🍒希望可以適合四季使用 妝感自然偏霧 不治痘
還有❗️❗️不要暗沉❗️❗️
（之前用Loréal 10小時下來臉直接黑兩階😫）

1.植村秀''')

In [15]:
kw

,0,tf,metion,idf,tfidf
2,專櫃,0.035088,1,2.056905,0.072172
2,植村秀,0.035088,1,2.056905,0.072172
2,粉底液,0.035088,3,1.579784,0.055431
1,遲遲,0.017544,1,2.056905,0.036086
1,Lor,0.017544,1,2.056905,0.036086
1,暗沉,0.017544,1,2.056905,0.036086
1,治痘,0.017544,1,2.056905,0.036086
1,偏霧,0.017544,1,2.056905,0.036086
1,痘疤,0.017544,1,2.056905,0.036086
1,泛紅,0.017544,1,2.056905,0.036086


In [22]:
topic_word = []
with open('topic.txt','r',encoding='utf-8') as file:
    for w in file:
        w = w.strip()
        topic_word.append(w)
        

In [25]:
kw['score'] = kw[0].apply(lambda x : 10 if x in topic_word else 1)

In [27]:
kw['output'] = kw['tfidf']*kw['score']

In [29]:
kw.sort_values('output')

,0,tf,metion,idf,tfidf,score,output
1,自己,0.017544,59,0.286053,0.005018,1,0.005018
1,可以,0.017544,54,0.324511,0.005693,1,0.005693
1,所以,0.017544,42,0.433656,0.007608,1,0.007608
1,不要,0.017544,42,0.433656,0.007608,1,0.007608
1,10,0.017544,35,0.512837,0.008997,1,0.008997
1,47,0.017544,31,0.565543,0.009922,1,0.009922
2,大家,0.035088,59,0.286053,0.010037,1,0.010037
1,23,0.017544,28,0.609747,0.010697,1,0.010697
1,希望,0.017544,28,0.609747,0.010697,1,0.010697
1,有一,0.017544,26,0.641932,0.011262,1,0.011262
